In [ ]:
import cdsapi
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.vectorized
import os

# Crea una cartella per salvare i file NetCDF scaricati
output_dir = "Italia_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Imposta il client CDS API
client = cdsapi.Client(url='https://ads.atmosphere.copernicus.eu/api',
                       key='6d4e1c06-ba15-4e08-a729-9f43ce76a3c1')

# Parametri fissi per la richiesta
dataset = "cams-europe-air-quality-forecasts"
# Lista di tutte le ore della giornata (dalle 00:00 alle 23:00)
hours = [f"{i:02d}:00" for i in range(24)]
# Area geografica per l'Italia: [nord, ovest, sud, est]
area = [47.1, 6.6, 36.6, 18.5]

# Suddividi l'anno 2024 in periodi mensili
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
for m in months:
    start_date = m.strftime("%Y-%m-%d")
    end_date = (m + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")
    
    request = {
        'variable': [ 
            'particulate_matter_2.5um', 
            'particulate_matter_10um',
            'carbon_monoxide',
            'nitrogen_dioxide',
            'sulphur_dioxide'
        ],
        'model': ['ensemble'],
        'level': ['0'],
        'date': [f'{start_date}/{end_date}'],
        'type': ['analysis'],
        'time': hours,  # tutte le ore della giornata
        'leadtime_hour': ['0'],
        'data_format': 'netcdf',
        'area': area
    }
    
    filename = os.path.join(output_dir, f"Italia_{m.strftime('%Y_%m')}.nc")
    print(f"Recupero dati dal {start_date} al {end_date}...")
    client.retrieve(dataset, request).download(filename)

# Aggrega tutti i file NetCDF in un unico dataset
nc_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".nc")]
combined_data = xr.open_mfdataset(nc_files, combine='by_coords')

# Funzione per aggregare i dati per regioni NUTS-3 (o livello desiderato) usando la media
def aggregate_by_nuts3(dataset, nuts3_gdf, variable_name='pm10_conc'):
    # Estrazione delle coordinate dal dataset
    times = dataset.time.values
    levels = dataset.level.values
    lats = dataset.latitude.values
    lons = dataset.longitude.values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    results = {
        'time': [],
        'level': [],
        'nuts3_id': [],
        'nuts3_name': [],
        f'mean_{variable_name}': []
    }
    
    da = dataset[variable_name]
    
    # Itera per ogni combinazione di tempo (ora) e livello
    for t_idx, t in enumerate(times):
        for l_idx, l in enumerate(levels):
            data_slice = da.isel(time=t_idx, level=l_idx).values
            # Per ciascuna regione (NUTS-3)
            for _, region in nuts3_gdf.iterrows():
                nuts3_id = region['NUTS_ID']
                nuts3_name = region['NUTS_NAME']
                region_geom = region.geometry
                
                mask = shapely.vectorized.contains(region_geom, lon_grid, lat_grid)
                mean_value = np.nanmean(data_slice[mask]) if np.any(mask) else np.nan
                
                results['time'].append(t)
                results['level'].append(l)
                results['nuts3_id'].append(nuts3_id)
                results['nuts3_name'].append(nuts3_name)
                results[f'mean_{variable_name}'].append(mean_value)
    
    return pd.DataFrame(results)

# Carica il file geojson con i confini NUTS e filtra per l'Italia
nuts = gpd.read_file("NUTS_RG_20M_2024_4326.geojson")
nuts = nuts[nuts["CNTR_CODE"] == "IT"]

# Aggrega i dati per ciascuna variabile
res_pm10 = aggregate_by_nuts3(combined_data, nuts, 'pm10_conc')
res_pm25 = aggregate_by_nuts3(combined_data, nuts, 'pm2p5_conc')
res_co   = aggregate_by_nuts3(combined_data, nuts, 'co_conc')
res_no2  = aggregate_by_nuts3(combined_data, nuts, 'no2_conc')
res_so2  = aggregate_by_nuts3(combined_data, nuts, 'so2_conc')

In [ ]:
import cdsapi
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.vectorized
import os

# Funzione per aggregare i dati per regioni NUTS-3 (o livello desiderato) usando la media
def aggregate_by_nuts3(dataset, nuts3_gdf, variable_name='pm10_conc'):
    times = dataset.time.values
    levels = dataset.level.values
    lats = dataset.latitude.values
    lons = dataset.longitude.values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    results = {
        'time': [],
        'level': [],
        'nuts3_id': [],
        'nuts3_name': [],
        f'mean_{variable_name}': []
    }
    
    da = dataset[variable_name]
    
    for t_idx, t in enumerate(times):
        for l_idx, l in enumerate(levels):
            data_slice = da.isel(time=t_idx, level=l_idx).values
            for _, region in nuts3_gdf.iterrows():
                nuts3_id = region['NUTS_ID']
                nuts3_name = region['NUTS_NAME']
                region_geom = region.geometry
                mask = shapely.vectorized.contains(region_geom, lon_grid, lat_grid)
                mean_value = np.nanmean(data_slice[mask]) if np.any(mask) else np.nan
                
                results['time'].append(t)
                results['level'].append(l)
                results['nuts3_id'].append(nuts3_id)
                results['nuts3_name'].append(nuts3_name)
                results[f'mean_{variable_name}'].append(mean_value)
    
    return pd.DataFrame(results)

# Funzione per creare una pivot table giornaliera
def create_daily_pivot_and_format(res, var_prefix):
    df = res.pivot_table(
        index='nuts3_name',
        columns='time',
        values=f'mean_{var_prefix}_conc',
        aggfunc='mean'
    )
    # Se la variabile time è già in formato datetime, questo passaggio non è necessario.
    # In caso contrario, bisogna adattarlo in base alla codifica temporale presente nei file .nc.
    try:
        df.columns = pd.to_datetime(df.columns)
    except Exception:
        baseline = pd.Timestamp("2024-01-01")
        df.columns = baseline + pd.to_timedelta(df.columns)
    
    # Raggruppa per data (troncando l'orario) e calcola la media giornaliera
    df_daily = df.groupby(df.columns.date, axis=1).mean()
    df_daily.columns = [pd.Timestamp(d).strftime('%d/%m/%Y') for d in df_daily.columns]
    return df_daily

# Cartella contenente i file .nc per ciascun mese
output_dir = "Italia_data"
nc_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".nc")]

# Carica il file geojson con i confini NUTS e filtra per l'Italia
nuts = gpd.read_file("NUTS_RG_20M_2024_4326.geojson")
nuts = nuts[nuts["CNTR_CODE"] == "IT"]

# Elabora ogni file .nc separatamente
for file in nc_files:
    print(f"Elaborazione di {file}...")
    ds = xr.open_dataset(file)
    
    # Aggrega per ciascuna variabile
    res_pm10 = aggregate_by_nuts3(ds, nuts, 'pm10_conc')
    res_pm25 = aggregate_by_nuts3(ds, nuts, 'pm2p5_conc')
    res_co   = aggregate_by_nuts3(ds, nuts, 'co_conc')
    res_no2  = aggregate_by_nuts3(ds, nuts, 'no2_conc')
    res_so2  = aggregate_by_nuts3(ds, nuts, 'so2_conc')
    
    # Crea le tabelle pivot per ogni variabile con media giornaliera
    df_pm25 = create_daily_pivot_and_format(res_pm25, 'pm2p5')
    df_pm10 = create_daily_pivot_and_format(res_pm10, 'pm10')
    df_co   = create_daily_pivot_and_format(res_co, 'co')
    df_no2  = create_daily_pivot_and_format(res_no2, 'no2')
    df_so2  = create_daily_pivot_and_format(res_so2, 'so2')
    
    # Estrae informazioni per denominare il file CSV (ad es. Italia_2024_01.nc)
    month_info = os.path.basename(file).replace("Italia_", "").replace(".nc", "")
    
    # Salva i DataFrame in file CSV per il mese corrente
    df_pm25.to_csv(f"Italia_pm25_daily_{month_info}.csv")
    df_pm10.to_csv(f"Italia_pm10_daily_{month_info}.csv")
    df_co.to_csv(f"Italia_co_daily_{month_info}.csv")
    df_no2.to_csv(f"Italia_no2_daily_{month_info}.csv")
    df_so2.to_csv(f"Italia_so2_daily_{month_info}.csv")


In [ ]:
import glob
import pandas as pd
import re

# Lista degli inquinanti da elaborare
pollutants = ['pm25', 'pm10', 'co', 'no2', 'so2']

for pol in pollutants:
    # Trova tutti i file CSV per l'inquinante corrente
    csv_files = glob.glob(f"Dati_italia_media/Italia_{pol}_daily_*.csv")
    df_list = []
    
    for f in csv_files:
        df = pd.read_csv(f, index_col=0)
        
        # Estrai anno e mese dal nome del file, es: "Italia_pm25_daily_2024_03.csv"
        match = re.search(r"(\d{4})_(\d{2})", f)
        if match:
            year = int(match.group(1))
            month = int(match.group(2))
            # Genera un range di date a partire dal primo giorno del mese, con il numero di giorni pari al numero di colonne
            num_days = len(df.columns)
            new_dates = pd.date_range(start=f"{year}-{month:02d}-01", periods=num_days, freq='D')
            # Rinomina le colonne con il formato 'dd/mm/YYYY'
            df.columns = new_dates.strftime('%d/%m/%Y')
        else:
            print(f"Attenzione: impossibile estrarre anno/mese da {f}")
        df_list.append(df)
    
    # Concatena i DataFrame lungo le colonne
    df_all = pd.concat(df_list, axis=1)
    
    # Ordina le colonne in ordine temporale:
    # Converte le colonne in datetime, le ordina e poi riassegna il formato 'dd/mm/YYYY'
    sorted_cols = sorted(df_all.columns, key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    df_all = df_all.loc[:, sorted_cols]
    
    # Salva il risultato in un CSV per l'inquinante corrente
    df_all.to_csv(f"Italia_{pol}_daily_all.csv")


In [ ]:
import cdsapi
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.vectorized
import os

# Crea una cartella per salvare i file NetCDF scaricati
output_dir = "Belgio_data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Imposta il client CDS API
client = cdsapi.Client(url='https://ads.atmosphere.copernicus.eu/api',
                       key='6d4e1c06-ba15-4e08-a729-9f43ce76a3c1')

# Parametri fissi
dataset = "cams-europe-air-quality-forecasts"
# Lista di tutte le ore della giornata
hours = [f"{i:02d}:00" for i in range(24)]
# Area geografica approssimativa per il Belgio [nord, ovest, sud, est]
area = [51.5, 2.5, 49.5, 6.4]

# Ciclo per ogni mese del 2024 (ad esempio, suddividendo per mese)
months = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
for m in months:
    start_date = m.strftime("%Y-%m-%d")
    end_date = (m + pd.offsets.MonthEnd(0)).strftime("%Y-%m-%d")
    
    request = {
        'variable': [ 
            'particulate_matter_2.5um', 
            'particulate_matter_10um',
            'carbon_monoxide',
            'nitrogen_dioxide',
            'sulphur_dioxide'
        ],
        'model': ['ensemble'],
        'level': ['0'],
        'date': [f'{start_date}/{end_date}'],
        'type': ['analysis'],
        'time': hours,  # tutti gli orari della giornata
        'leadtime_hour': ['0'],
        'data_format': 'netcdf',
        'area': area
    }
    
    filename = os.path.join(output_dir, f"Belgio_{m.strftime('%Y_%m')}.nc")
    print(f"Recupero dati dal {start_date} al {end_date}...")
    client.retrieve(dataset, request).download(filename)

# Aggrega tutti i file NetCDF in un unico dataset
nc_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".nc")]
combined_data = xr.open_mfdataset(nc_files, combine='by_coords')

# Funzione per aggregare i dati per regioni NUTS-3 usando la media
def aggregate_by_nuts3(dataset, nuts3_gdf, variable_name='pm10_conc'):
    times = dataset.time.values
    levels = dataset.level.values
    lats = dataset.latitude.values
    lons = dataset.longitude.values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    results = {
        'time': [],
        'level': [],
        'nuts3_id': [],
        'nuts3_name': [],
        f'mean_{variable_name}': []
    }
    
    da = dataset[variable_name]
    
    # Per ogni combinazione di tempo (ora) e livello
    for t_idx, t in enumerate(times):
        for l_idx, l in enumerate(levels):
            data_slice = da.isel(time=t_idx, level=l_idx).values
            # Per ciascuna regione NUTS-3
            for _, region in nuts3_gdf.iterrows():
                nuts3_id = region['NUTS_ID']
                nuts3_name = region['NUTS_NAME']
                region_geom = region.geometry
                
                # Crea la maschera dei punti interni alla geometria
                mask = shapely.vectorized.contains(region_geom, lon_grid, lat_grid)
                mean_value = np.nanmean(data_slice[mask]) if np.any(mask) else np.nan
                
                results['time'].append(t)
                results['level'].append(l)
                results['nuts3_id'].append(nuts3_id)
                results['nuts3_name'].append(nuts3_name)
                results[f'mean_{variable_name}'].append(mean_value)
                
    return pd.DataFrame(results)

# Carica il file geojson con i confini NUTS e filtra per il Belgio
nuts = gpd.read_file("NUTS_RG_20M_2024_4326.geojson")
nuts = nuts[nuts["CNTR_CODE"] == "BE"]

# Aggrega i dati per ciascuna variabile
res_pm10 = aggregate_by_nuts3(combined_data, nuts, 'pm10_conc')
res_pm25 = aggregate_by_nuts3(combined_data, nuts, 'pm2p5_conc')
res_co   = aggregate_by_nuts3(combined_data, nuts, 'co_conc')
res_no2  = aggregate_by_nuts3(combined_data, nuts, 'no2_conc')
res_so2  = aggregate_by_nuts3(combined_data, nuts, 'so2_conc')

In [ ]:
import cdsapi
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely.vectorized
import os

# Funzione per aggregare i dati per regioni NUTS-3 (o livello desiderato) usando la media
def aggregate_by_nuts3(dataset, nuts3_gdf, variable_name='pm10_conc'):
    times = dataset.time.values
    levels = dataset.level.values
    lats = dataset.latitude.values
    lons = dataset.longitude.values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    
    results = {
        'time': [],
        'level': [],
        'nuts3_id': [],
        'nuts3_name': [],
        f'mean_{variable_name}': []
    }
    
    da = dataset[variable_name]
    
    for t_idx, t in enumerate(times):
        for l_idx, l in enumerate(levels):
            data_slice = da.isel(time=t_idx, level=l_idx).values
            for _, region in nuts3_gdf.iterrows():
                nuts3_id = region['NUTS_ID']
                nuts3_name = region['NUTS_NAME']
                region_geom = region.geometry
                mask = shapely.vectorized.contains(region_geom, lon_grid, lat_grid)
                mean_value = np.nanmean(data_slice[mask]) if np.any(mask) else np.nan
                
                results['time'].append(t)
                results['level'].append(l)
                results['nuts3_id'].append(nuts3_id)
                results['nuts3_name'].append(nuts3_name)
                results[f'mean_{variable_name}'].append(mean_value)
    
    return pd.DataFrame(results)

# Funzione per creare una pivot table giornaliera
def create_daily_pivot_and_format(res, var_prefix):
    df = res.pivot_table(
        index='nuts3_name',
        columns='time',
        values=f'mean_{var_prefix}_conc',
        aggfunc='mean'
    )
    # Se la variabile time è già in formato datetime, questo passaggio non è necessario.
    # In caso contrario, bisogna adattarlo in base alla codifica temporale presente nei file .nc.
    try:
        df.columns = pd.to_datetime(df.columns)
    except Exception:
        baseline = pd.Timestamp("2024-01-01")
        df.columns = baseline + pd.to_timedelta(df.columns)
    
    # Raggruppa per data (troncando l'orario) e calcola la media giornaliera
    df_daily = df.groupby(df.columns.date, axis=1).mean()
    df_daily.columns = [pd.Timestamp(d).strftime('%d/%m/%Y') for d in df_daily.columns]
    return df_daily

# Cartella contenente i file .nc per ciascun mese per il Belgio
output_dir = "Belgio_data"
nc_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".nc")]

# Carica il file geojson con i confini NUTS e filtra per il Belgio
nuts = gpd.read_file("NUTS_RG_20M_2024_4326.geojson")
nuts = nuts[nuts["CNTR_CODE"] == "BE"]

# Elabora ogni file .nc separatamente
for file in nc_files:
    print(f"Elaborazione di {file}...")
    ds = xr.open_dataset(file)
    
    # Aggrega per ciascuna variabile
    res_pm10 = aggregate_by_nuts3(ds, nuts, 'pm10_conc')
    res_pm25 = aggregate_by_nuts3(ds, nuts, 'pm2p5_conc')
    res_co   = aggregate_by_nuts3(ds, nuts, 'co_conc')
    res_no2  = aggregate_by_nuts3(ds, nuts, 'no2_conc')
    res_so2  = aggregate_by_nuts3(ds, nuts, 'so2_conc')
    
    # Crea le tabelle pivot per ogni variabile con media giornaliera
    df_pm25 = create_daily_pivot_and_format(res_pm25, 'pm2p5')
    df_pm10 = create_daily_pivot_and_format(res_pm10, 'pm10')
    df_co   = create_daily_pivot_and_format(res_co, 'co')
    df_no2  = create_daily_pivot_and_format(res_no2, 'no2')
    df_so2  = create_daily_pivot_and_format(res_so2, 'so2')
    
    # Estrae informazioni per denominare il file CSV (ad es. Belgio_2024_01.nc)
    month_info = os.path.basename(file).replace("Belgio_", "").replace(".nc", "")
    
    # Salva i DataFrame in file CSV per il mese corrente
    df_pm25.to_csv(f"Belgio_pm25_daily_{month_info}.csv")
    df_pm10.to_csv(f"Belgio_pm10_daily_{month_info}.csv")
    df_co.to_csv(f"Belgio_co_daily_{month_info}.csv")
    df_no2.to_csv(f"Belgio_no2_daily_{month_info}.csv")
    df_so2.to_csv(f"Belgio_so2_daily_{month_info}.csv")


In [ ]:
import glob
import pandas as pd
import re

# Lista degli inquinanti da elaborare
pollutants = ['pm25', 'pm10', 'co', 'no2', 'so2']

for pol in pollutants:
    # Trova tutti i file CSV per l'inquinante corrente nella cartella Belgio_data
    csv_files = glob.glob(f"Dati_belgio_media/Belgio_{pol}_daily_*.csv")
    df_list = []
    
    for f in csv_files:
        df = pd.read_csv(f, index_col=0)
        
        # Estrai anno e mese dal nome del file, es: "Belgio_pm25_daily_2024_03.csv"
        match = re.search(r"(\d{4})_(\d{2})", f)
        if match:
            year = int(match.group(1))
            month = int(match.group(2))
            # Genera un range di date a partire dal primo giorno del mese, con il numero di giorni pari al numero di colonne
            num_days = len(df.columns)
            new_dates = pd.date_range(start=f"{year}-{month:02d}-01", periods=num_days, freq='D')
            # Rinomina le colonne con il formato 'dd/mm/YYYY'
            df.columns = new_dates.strftime('%d/%m/%Y')
        else:
            print(f"Attenzione: impossibile estrarre anno/mese da {f}")
        df_list.append(df)
    
    # Concatena i DataFrame lungo le colonne
    df_all = pd.concat(df_list, axis=1)
    
    # Ordina le colonne in ordine temporale:
    sorted_cols = sorted(df_all.columns, key=lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
    df_all = df_all.loc[:, sorted_cols]
    
    # Salva il risultato in un CSV per l'inquinante corrente
    df_all.to_csv(f"Belgio_{pol}_daily_all.csv")
